In [1]:
# 데이터 크롤링
import requests
import csv
import json

# API 주소(요리명과 조리 방법)를 통해 데이터 긁어와 csv로 저장하는 코드
url = "http://openapi.foodsafetykorea.go.kr/api/439dfa58a5bd4e228198/COOKRCP01/json/1001/2000"

response = requests.get(url)
data = response.json()

recipes = data['COOKRCP01']['row']

with open('recipe2.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)

    headers = recipes[0].keys()
    writer.writerow(headers)

    for recipe in recipes:
        writer.writerow(recipe.values())

In [2]:
# 데이터 정제
import pandas as pd

data = pd.read_csv('ingredient.csv', encoding='cp949')

# 대표 식품명 기준 1개만 남기고 나머지 삭제
data = data.drop_duplicates(subset='대표식품명', keep='first')

# 새로운 CSV로 저장
data.to_csv('new_ingredient.csv', index=False, encoding='utf-8')

In [3]:
# 한 번 더 정제
import pandas as pd

data = pd.read_csv('new_ingredient.csv')

# '식품명' 열의 값 정제
data['식품명'] = data['식품명'].str.split('_').str[0] # _과 그 이후 단어 삭제
data['식품명'] = data['식품명'].str.split('(').str[0] # (와 그 이후 단어 삭제

data.to_csv('final_ingredient.csv', index=False, encoding='utf-8')

In [2]:
# 진짜 마지막 정제  
import pandas as pd

data = pd.read_csv('final_ingredient.csv')

# '식품명'에서 ~류 삭제
data['식품명'] = data['식품명'].str.split('류').str[0] # ~류 삭제

data.to_csv('real_final_ingredient.csv', index=False, encoding='utf-8')

In [ ]:
# recipe 정제
import pandas as pd

df = pd.read_csv('recipe.csv')

cut_list = []

for idx, row in df.iterrows():
    if pd.isnull(row['RCP_PARTS_DTLS']):
        continue # NAN 값이면 건너뛰기
    first_char = row['RCP_PARTS_DTLS'][0] # 첫 번째 문자 추출
    if first_char in [':', ')', ']', '(', '[']: # 첫 번째 문자가 특수문자인지 확인
        cut_list.append(first_char) # 첫 번째 문자를 cut_list에 추가
        df.loc[idx, 'RCP_PARTS_DTLS'] = row['RCP_PARTS_DTLS'][1:].strip() # 첫 번째 문자 제거

# 잘라낸 내용을 따로 csv로 저장
cut_df = pd.DataFrame(cut_list, columns=['cut'])
cut_df.to_csv('cut.csv', index=False, encoding='utf-8')

# 나머지 내용을 새로운 csv로 저장
df.to_csv('new_recipe.csv', index=False, encoding='utf-8')


In [21]:
# 세미콜론 가준으로 MAIN, SUB 등으로 쪼개기
import pandas as pd
import re

def split_data(data):
    # NaN 값이나 숫자 값을 검사
    if pd.isna(data) or isinstance(data, (int, float)):
        return {"RCP_MAIN": "", "RCP_SUB1": "", "RCP_SUB2": "", "RCP_SUB3": "", "RCP_SUB4": "", "RCP_SUB5": ""}
    
    data = re.sub(r'\b\w+\s*:', ':', data)
    
    parts = data.split(':')
    result = {"RCP_MAIN": "", "RCP_SUB1": "", "RCP_SUB2": "", "RCP_SUB3": "", "RCP_SUB4": "", "RCP_SUB5": ""}
    for i, part in enumerate(parts):
        if i == 0:
            result["RCP_MAIN"] = part.strip()
        elif i == 1:
            result["RCP_SUB1"] = part.strip()
        elif i == 2:
            result["RCP_SUB2"] = part.strip()
        elif i == 3:
            result["RCP_SUB3"] = part.strip()
        elif i == 4:
            result["RCP_SUB4"] = part.strip()
        elif i == 5:
            result["RCP_SUB5"] = part.strip()
    return result

df = pd.read_csv('recipe.csv')

df_cut = df['RCP_PARTS_DTLS'].apply(split_data).apply(pd.Series)

df_cut.to_csv('cut.csv', index=False, encoding='utf-8')

In [33]:
import pandas as pd
import re

def split_data(data):
    # NaN 값이나 숫자 값을 검사
    if pd.isna(data) or isinstance(data, (int, float)):
        return {"RCP_MAIN": "", "RCP_SUB1": "", "RCP_SUB2": "", "RCP_SUB3": "", "RCP_SUB4": "", "RCP_SUB5": ""}

    data = re.sub(r'\b\w+\s*:', ':', data)

    parts = data.split(':')
    result = {"RCP_MAIN": "", "RCP_SUB1": "", "RCP_SUB2": "", "RCP_SUB3": "", "RCP_SUB4": "", "RCP_SUB5": ""}
    for i, part in enumerate(parts):
        if i == 0:
            result["RCP_MAIN"] = part.strip()
        elif i == 1:
            result["RCP_SUB1"] = part.strip()
        elif i == 2:
            result["RCP_SUB2"] = part.strip()
        elif i == 3:
            result["RCP_SUB3"] = part.strip()
        elif i == 4:
            result["RCP_SUB4"] = part.strip()
        elif i == 5:
            result["RCP_SUB5"] = part.strip()
    return result

df = pd.read_csv('recipe.csv')

df_cut = df['RCP_PARTS_DTLS'].apply(split_data).apply(pd.Series)

# 기존 df에 새로 생성된 df_cut을 결합
df = pd.concat([df, df_cut], axis=1)

df.to_csv('cut.csv', index=False, encoding='utf-8')


In [35]:
# 특정 열에 값이 비어있는 지 찾고, 그 위치 반환하기
import pandas as pd

# cut.csv 파일 읽기
df_cut = pd.read_csv('new_recipe.csv')

# 'RCP_SUB3' 열에서 값이 존재하지 않는 행 찾기
non_existing_values = df_cut['RCP_MAIN'].isna()

# 값이 존재하지 않는 행의 위치 찾기
rows_without_value = non_existing_values[non_existing_values].index

# 위치 정보 출력 (1부터 시작하도록 +1)
print(f"'RCP_MAIN' 열에서 값이 존재하지 않는 행의 위치: {rows_without_value + 1}")


'RCP_MAIN' 열에서 값이 존재하지 않는 행의 위치: Index([], dtype='int64')


In [36]:
# RCP 방법 열에서 특정 특수문자(?, -)와 그 특수문자와 붙은 단어 삭제하기
import pandas as pd
import re 

df = pd.read_csv('new_recipe.csv')

# 'RCP_MAIN', 'RCP_SUB1', 'RCP_SUB2', 'RCP_SUB3', 'RCP_SUB4', 'RCP_SUB5' 열에서 특수문자(`?`, `-`)와 그 뒤의 모든 문자 삭제
columns = ['RCP_MAIN', 'RCP_SUB1', 'RCP_SUB2', 'RCP_SUB3', 'RCP_SUB4', 'RCP_SUB5']
for column in columns:
    df[column] = df[column].replace(to_replace=r'[?-].*', value='', regex=True)

df.to_csv('recipe_new.csv', index=False, encoding='utf-8')


In [41]:
import pandas as pd

# recut.csv 파일 읽기
df = pd.read_csv('recipe2.csv')

# 여러 열을 하나로 합치기
df['RCP_PARTS_DTLS'] = df[['RCP_MAIN', 'RCP_SUB1', 'RCP_SUB2', 'RCP_SUB3', 'RCP_SUB4', 'RCP_SUB5']].apply(lambda row: ':'.join(row.values.astype(str)), axis=1)

# ':nan' 문자열 삭제
df['RCP_PARTS_DTLS'] = df['RCP_PARTS_DTLS'].str.replace(':nan', '')

# ':' 문자열을 ',' 문자열로 치환
df['RCP_PARTS_DTLS'] = df['RCP_PARTS_DTLS'].str.replace(':', ', ')

# 'RCP_MAIN', 'RCP_SUB1', 'RCP_SUB2', 'RCP_SUB3', 'RCP_SUB4', 'RCP_SUB5' 열 삭제
df = df.drop(columns=['RCP_MAIN', 'RCP_SUB1', 'RCP_SUB2', 'RCP_SUB3', 'RCP_SUB4', 'RCP_SUB5'])

# 결과를 recut.csv 파일에 저장
df.to_csv('cut.csv', index=False, encoding='utf-8')


In [51]:
# csv 특정 열에 값이 몇 개 비어있는 지 세고, 그 위치 반환하기
import pandas as pd

# cut.csv 파일 읽기
df_cut = pd.read_csv('recipe.csv')

# 'RCP_PARTS_DTLS' 열에서 값이 존재하지 않는 행 찾기
non_existing_values = df_cut['RCP_PARTS_DTLS'].isna()

# 값이 존재하지 않는 행의 위치 찾기
rows_without_value = non_existing_values[non_existing_values].index

# 위치 정보 출력 (1부터 시작하도록 +1)
print(f"'RCP_PARTS_DTLS' 열에서 값이 존재하지 않는 행의 위치: {rows_without_value + 1}")

'RCP_PARTS_DTLS' 열에서 값이 존재하지 않는 행의 위치: Index([], dtype='int64')


In [57]:
# 콤마 정제 1탄: g 뒤에 콤마 없으면 붙이기
import pandas as pd
import re

# recut.csv 파일 읽기
df = pd.read_csv('CUT.csv')

# 줄바꿈 문자가 있는 문자열 찾기
df['RCP_PARTS_DTLS'] = df['RCP_PARTS_DTLS'].apply(lambda x: re.sub(r'g(?![:,])', 'g,', x) if pd.notnull(x) else x)

# 결과를 recut.csv 파일에 저장
df.to_csv('recut.csv', index=False, encoding='utf-8')


In [74]:
import pandas as pd
import re

# recut.csv 파일 읽기
df = pd.read_csv('recipe.csv')

# 문장의 맨 앞에 콤마가 있는 경우 삭제
df['RCP_PARTS_DTLS'] = df['RCP_PARTS_DTLS'].apply(lambda x: re.sub(r'^,', '', x) if pd.notnull(x) else x)

# g 뒤에 콤마가 아닌 다른 문자가 오면 g 뒤에 콤마를 붙이되, 띄어쓰기와 줄바꿈은 제외
df['RCP_PARTS_DTLS'] = df['RCP_PARTS_DTLS'].apply(lambda x: re.sub(r'g(?![:,\s])', 'g,', x) if pd.notnull(x) else x)

# 결과를 recut.csv 파일에 저장
df.to_csv('cut.csv', index=False, encoding='utf-8')


In [77]:
import pandas as pd
import re

# recipe.csv 파일 읽기
df = pd.read_csv('recipe.csv')

# RCP_PARTS_DTLS에 있는 콤마를 전부 삭제
df['RCP_PARTS_DTLS'] = df['RCP_PARTS_DTLS'].apply(lambda x: re.sub(',', '', x) if pd.notnull(x) else x)

# g 뒤에 콤마를 다 붙임
df['RCP_PARTS_DTLS'] = df['RCP_PARTS_DTLS'].apply(lambda x: re.sub('g', 'g,', x) if pd.notnull(x) else x)

# 문장에 맨 끝에 오는 콤마는 삭제
df['RCP_PARTS_DTLS'] = df['RCP_PARTS_DTLS'].apply(lambda x: re.sub(',$', '', x) if pd.notnull(x) else x)

# 결과를 recipe.csv 파일에 저장
df.to_csv('cut.csv', index=False, encoding='utf-8')


In [8]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
# API 요청
url = "https://apis.data.go.kr/B551182/diseaseInfoService/getDissNameCodeList?serviceKey=e4ZK69MusxTk8Rk0s0US%2Bc9uLgmYzyvmC4%2F4cFXVlWm6emK8KIQDH8TA1I8zv173L1YHFn4SU8P9cd217BWT5g%3D%3D&numOfRows=3000&pageNo=1&year=2022&sickCd=A00&sickType=1&medTp=1"

response = requests.get(url)

# BeautifulSoup으로 파싱
soup = BeautifulSoup(response.text, 'lxml')

# 데이터 추출
items = soup.find_all('item')
data = []
for item in items:
    sickCd = item.find('sickcd').text
    sickNm = item.find('sicknm').text
    data.append([sickCd, sickNm])

# 데이터 프레임 생성
df = pd.DataFrame(data, columns=['sickCd', 'sickNm'])

# CSV 파일로 저장
df.to_csv('disease.csv', index=False, encoding='utf-8')

C:\Users\jhy97\AppData\Local\Temp\ipykernel_4392\4098798430.py:10: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  soup = BeautifulSoup(response.text, 'lxml')
